In [1]:
from aocd import submit
import numpy as np
import re

In [2]:
YYYY=2021
DD=19

In [15]:
def load(filename):
    with open(filename,'r') as f:
        scanners = []
        ss = []
        for s in f:
            if s == "\n":
                continue
            if s.startswith("---"):
                if ss: 
                    scanners.append(np.array(ss))
                    ss = []
            else:
                ss.append([int(n) for n in s.rstrip("\n").split(",")])
        else:
            scanners.append(np.array(ss))
    return scanners
test1 = load("test1.dat")
test2 = load("test2.dat")
test1

[array([[0, 2],
        [4, 1],
        [3, 3]]),
 array([[-1, -1],
        [-5,  0],
        [-2,  1]])]

In [13]:
data_in = load("input.dat")

# Teil a

In [641]:
def rotate(scanner, rot):
    r = np.array(scanner, copy=True)  
    for dim, rr in enumerate(f"{rot:03b}"):
        if rr == "1":
            r[:,dim] *= -1 
    return r

def flip(scanner, f):
    r = np.array(scanner, copy=True)  
    return r[:,f]

def distances(scanner):
    dim = scanner.shape[0]
    dist = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(i+1,dim):
            dist[i,j] = np.linalg.norm(scanner[i]-scanner[j]).round(2)
            dist[j,i] = dist [i,j]
    return dist

def compare(s1, s2, min_overlap=12):
    d1=distances(s1)
    d2=distances(s2)
    for i, di in enumerate(d1):
        for j, dj in enumerate(d2):
            #print(np.intersect1d(i,j))
            intersection, ii, jj = np.intersect1d(di,dj, return_indices=True)
            if intersection.shape[0] >= min_overlap:
                #print(f"{i} {j}")
                return ii, jj
    return None, None

def all_equal(arr):
    # Check if all value in 2D array are equal
    u, c = np.unique(arr, return_counts = True, axis=0)
    if c.max() < 12:
        return None
    return u[c == c.max()]
    
def transform(s1, s2, idx1, idx2):
    for r in range(9):
        tmp = rotate(s2, r)
        for f in [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1),(2,1,0)]:
            tmp2 = flip(tmp, f)
            a = all_equal(tmp2[idx2]-s1[idx1])
            if a is not None:
                return tmp2-a, a
    return None
    
def merge(s1, s2):
    s = np.vstack((s1, s2))
    return np.unique(s, axis=0)
    
def solution_a(data):
    s=data[0]
    merge_list=[0]
    scanners = [np.array([0,0,0])]
    mem = 0
    for _ in range(10):
        for i, s1 in enumerate(data):
            if i in merge_list:
                continue
            ii, jj = compare(s, s1)
            if ii is not None:
                t, spos = transform(s, s1, ii, jj) 
                scanners.append(spos)
                merge_list.append(i)
                s = merge(s,t)
        if s.shape[0] == mem:
            return s, scanners
        else: 
            mem = s.shape[0]
    return s, scanners



In [642]:
solution_a(test2)[0].shape[0]

79

In [635]:
submit(solution_a(data_in)[0].shape[0],part="a",day=DD, year=YYYY)

Part a already solved with same answer: 303


# Teil b

In [638]:
def solution_b(data):
    _, pos = solution_a(data)
    x = 0
    for i, p1 in enumerate(pos):
        for j,p2 in enumerate(pos):
            if i <= j:
                continue
            dist = np.abs(p1-p2).sum()
            if dist > x:
                x = dist
    return x

In [639]:
solution_b(test2)

3621

In [640]:
submit(solution_b(data_in),part="b",day=DD, year=YYYY)

Part b already solved with same answer: 9621
